# Creating Documents

In [9]:
from mongoengine import *

In [12]:
disconnect()

In [13]:
# Connect to the local MongoDB instance
localDb = connect(host='mongodb://localhost/contactData')

In [14]:
# Custom validator function for salary
def validate_salary(value):
    if value < 5000:
        raise ValidationError("Salary cannot be less than 5000")

In [15]:
# Define the document schema for the employee collection
class Employee(Document):
    first_name = StringField(required=True, max_length=100)
    last_name = StringField(required=True, max_length=100)
    phone = StringField(max_length=20, required=True)
    email = EmailField(required=True, unique=True)
    address = StringField(required=True)
    salary = DecimalField(precision=2, required=True, validation=validate_salary)

    meta = {'collection': 'employee'}  # Name of the collection in the database

    # Ensure no field is empty
    def clean(self):
        if not self.first_name:
            raise ValidationError("First name cannot be empty")
        if not self.last_name:
            raise ValidationError("Last name cannot be empty")
        if not self.phone:
            raise ValidationError("Phone cannot be empty")
        if not self.address:
            raise ValidationError("Address cannot be empty")

In [16]:
# Create an instance of the Employee document
new_employee = Employee(
    first_name="John",
    last_name="Doe",
    phone="123-456-7890",
    email="john.doe@example.com",
    address="123 Main St, City, Country",
    salary=6000.00  # Assuming salary is above the minimum required
)

try:
    # Save the document to the database
    new_employee.save()

    # Verify the insertion by fetching all documents in the employee collection
    for emp in Employee.objects:
        print(emp.first_name, emp.last_name, emp.phone, emp.email, emp.address, emp.salary)

except ValidationError as e:
    print(f"Validation Error: {e}")
except NotUniqueError:
    print("Email must be unique")
except Exception as e:
    print(f"Error: {e}")

John Doe 123-456-7890 john.doe@example.com 123 Main St, City, Country 6000.00
